In [1]:
!pip install PyPDF2
!pip install tqdm
!pip install -U sentence-transformers

!pip install protobuf==3.20
!pip install bitsandbytes
!python -m bitsandbytes

!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 121.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 142.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.2 MB/s eta

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

path = "davidkim205/komt-mistral-7b-v1"
tokenizer = AutoTokenizer.from_pretrained(path, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, low_cpu_mem_usage=True, device_map="auto")
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [54]:
system_prompt = """
RPACA 설치 가이드  1)RPACA 설치파일을 더블 클릭합니다. \n 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. \n 3)Install 버튼을 클릭합니다. \n 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.
RPACA 사용법  1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다.\n2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. \n상단 부분에는 선택한 폴더 경로가 표시됩니다\n3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다.4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다.\n5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 \n단계의 activity를 연결합니다. \n6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 \n진행되는지 확인합니다. (주모니터에서만 작동)\n7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.
Activity 이름이 Image로 끝날 경우 이미지 캡쳐하는 방법  1)이름이 Image로 끝나는 Activity를 클릭하면 Option 옆에 사진기 \n아이콘이 존재합니다. 해당아이콘을 클릭합니다. \n2)아이콘을 클릭하면 알파카가 최소화 되면서 원하는 영역을 드래그 앤 \n드롭으로 지정하면 해당 영역이 캡처됩니다. (예시 이미지에선 검색 \n아이콘을 지정)\n3)imagePath 부분에 이미지를 문자화 시킨 값이 들어가고, 옵션 \n하단부에 캡처한 이미지가 나타나고 작업을 완료합니다.
Activity 이름이 XPath로 끝날 경우 xpath 값 구하는 방법  1)크롬 브라우저에서 키보드의 F12 를 클릭합니다.\n2)이미지에 표시 된 부분을 클릭합니다. 3)브라우저 내에서 xpath를 구하고자 하는 곳에 마우스를 올리면 \n이미지와 같이 표시됩니다. \n4)마우스로 클릭하게 되면 이미지와 같이 색으로 하이라이트 되어 \n나타나게 됩니다. \n5)하이라이트 된 부분에 마우스 오른쪽 클릭하고 Copy – Copy full \nxpath를 클릭하면 xpath가 복사됩니다. \n6)옵션에서xpath부분에 ctrl + v 를 눌러 붙여넣어 줍니다.
Activities 기능 설명  openBrowser, openCMD, getURL, inputBrowserByXPath,  clickBrowserByXPath, inputBrowserByImage, clickBrowserByImage, clicks, hotkey, clickBrowserByImage, click, BrowserByImage, typingText,  switchingTap, scrollSearchByImage, trycatch, closeBrowser, if, getText, w
openBrowser  크롬 브라우저를 오픈합니다.\nOption\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
openCMD  명령 프롬프트를 실행합니다.\nOption\nadmin : true라고 입력 할 경우 관리자 권한으로 실행합니다. 그 외에는 일반 \n권한으로 실행합니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
getURL  브라우저 주소창에 주소를 입력합니다.\nOption\nurl : 주소창에 입력 할 주소입니다. (예시: www.infofla.com)\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
inputBrowserByXPath  브라우저 안에서 사용자가 내용을 입력 할 수 있는 부분(예시: 검색창)에 \n값을 입력합니다.\nOption\nxpath : 브라우저 내의 xpath를 입력합니다. (매뉴얼 맨 처음 부분 참조)\ninput : 입력하고 싶은 내용을 적습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
clickBrowserByXPath  브라우저 안의 특정 부분을 클릭합니다.Option\nxpath : 브라우저 내의 xpath를 입력합니다. (매뉴얼 맨 처음 부분 참조)\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
inputBrowserByImage  브라우저 안에서 사용자가 내용을 입력 할 수 있는 부분(예시: 검색창)에 \n값을 입력합니다.\ninputBrowserByXPath와는 다르게 캡처한 이미지를 기준으로 브라우저 내의 \n원하는 부분에 내용을 입력합니다.\n이미지 캡처는 옵션쪽의 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 \n원하는 영역을 지정하면 가능합니다.\nOption\nimagePath : 사진기 아이콘을 클릭하고 이미지를 캡처하였을 때 해당 \n이미지를 문자화 시킨 값이 자동으로 들어갑니다. (신경 안쓰셔도 됩니다.)\ninput : 입력하고 싶은 내용을 적습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.
clickBrowserByImage  브라우저 안의 특정 부분을 클릭합니다.\nclickBrowserByXPath와는 다르게 캡처한 이미지를 기준으로 브라우저 내의 \n원하는 부분을 클릭합니다.\n이미지 캡처는 옵션쪽의 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 \n원하는 영역을 지정하면 가능합니다.\nOption\nimagePath : 사진기 아이콘을 클릭하고 이미지를 캡처하였을 때 해당 \n이미지를 문자화 시킨 값이 자동으로 들어갑니다. (신경 안쓰셔도 됩니다.)\nclicks (숫자) : 원하는 클릭 횟수를 입력합니다.\ninterval (숫자) : 클릭 횟수 사이의 대기시간을 설정합니다. (예시: click이 \n2이고 interval이 1이면 한번 클릭 한 뒤 1초 대기 후 다시 한번 \n클릭합니다.)\nbutton : 마우스의 왼쪽 버튼으로 클릭할 것인지 오른쪽 버튼으로 클릭할 \n것인지 정합니다. (left 라고 적으면 왼쪽 클릭, right라고 적으면 오른쪽 클릭)\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.

[INST]반드시 prompt 내용만을 사용하여 '질문-답변'을 5개 이상 만들어주세요.[/INST]
"""

In [55]:
from transformers import TextStreamer, TextIteratorStreamer
from threading import Thread


# def stream(user_prompt, model, system_prompt):
def stream(model, system_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = f"""{system_prompt}"""
    # B_INST, E_INST = "[INST]", "[/INST]"
    # prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n{E_INST}"
    prompt = f"{system_prompt}"
    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)
    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=True)
    output = model.generate(**inputs, streamer=streamer, max_new_tokens=4096, use_cache=True)
    return_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    
    return return_output

In [56]:
result = stream(model, system_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



RPACA 설치 가이드  1)RPACA 설치파일을 더블 클릭합니다. 
 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. 
 3)Install 버튼을 클릭합니다. 
 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.
RPACA 사용법  1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다.
2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. 
상단 부분에는 선택한 폴더 경로가 표시됩니다
3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다.4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다.
5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 
단계의 activity를 연결합니다. 
6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 
진행되는지 확인합니다. (주모니터에서만 작동)
7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.
Activity 이름이 Image로 끝날 경우 이미지 캡쳐하는 방법  1)이름이 Image로 끝나는 Activity를 클릭하면 Option 옆에 사진기 
아이콘이 존재합니다. 해당아이콘을 클릭합니다. 
2)아이콘을 클릭하면 알파카가 최소화 되면서 원하는 영역을 드래그 앤 
드롭으로 지정하면 해당 영역이 캡처됩니다. (예시 이미지에선 검색 
아이콘을 지정)
3)imagePath 부분에 이미지를 문자화 시킨 값이 들어가고, 옵션 
하단부에 캡처한 이미지가 나타나고 작업을 완료합니다.
Activity 이름이 XPath로 끝날 경우 xpath 값 구하는 방법  1)크롬 브라우저에서 키보드의 F12 를 클릭합니다.
2)이미지에 표시 된 부분을 클릭합니다. 3)브라우저 내에서 xpath를 구하고자 하는 곳에 마우스를 올리면 
이미지와 같이 표시됩니다. 
4)마우스로 클릭하게 되면 이미지와 같이 색으로 하이라이트 되어 
나

In [5]:
result = stream("반드시 prompt만을 사용하여 짧은 질문과 답변 형태를 만들어줄 수 있나요? 순서에 대한 내용도 포함하여, 10개 이상 만들어주세요.", model, system_prompt)

Token indices sequence length is longer than the specified maximum sequence length for this model (2754 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. RPACA 설치 가이드
  - RPACA 설치파일을 더블 클릭합니다.
  - Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다.
  - Install 버튼을 클릭합니다.
  - 설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.

2. RPACA 사용법
  - RPACA 파일을 저장 및 

KeyboardInterrupt: 

In [16]:
result = stream("반드시 prompt만을 사용하여 짧은 질문과 답변 형태를 10개 이상 만들어주세요.", model, system_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. RPACA는 무엇인가요?
RPACA는 사용자가 원하는 작업을 자동화하기 위한 스크립트 파일입니다. 이 스크립트 파일은 크롬 브라우저를 통해 실행되며, 사용자가 원하는 작업을 수행하기 위해 필요한 명령어를 포함하고 있습니다.
2. RPACA를 설치하기 위해서는 어떤 단계를 거쳐야 하나요?
RPACA를 설치하기 위해서는 다음과 같은 단계를 거쳐야 합니다:
1) RPACA 설치파일을 더블 클릭합니다.
2) Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다.
3) Install 버튼을 클릭합니다.
4) 설치가 다 진행 될때까지 기다린 후 FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.
3. RPACA를 사용하기 위해서는 어떤 단계를 거쳐야 하나요?
RPACA를 사용하기 위해서는 다음과 같은 단계를 거쳐야 합니다:
1) RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다.
2) 해당 폴더 경로가 표시된 상단 부분에 있는 파일 리스트에 RPACA 파일이 존재한다면 이미지 캡처 및 클릭 등의 작업을 수행할 수 있습니다.
3) 필요한 작업을 수행하기 위해 필요한 명령어를 작성하고 실행 버튼을 클릭합니다.
4) 작업이 완료되면 결과를 확인할 수 있습니다.
4. RPACA를 사용하기 위해서는 어떤 명령어를 사용해야 하나요?
RPACA를 사용하기 위해서는 다양한 명령어를 사용할 수 있습니다. 예를 들어, 크롬 브라우저를 실행하는 명령어는 openBrowser 명령어를 사용할 수 있습니다. 또한, 브라우저 내에서 특정 작업을 수행하는 명령어도 제공됩니다. 이러한 명령어를 사용하여 원하는 작업을 자동화할 수 있습니다.
5. RPACA를 사용하기 위해서는 어떤 설정을 해야 하나요?
RPACA를 사용하기 위해서는 다음과 같은 설정을 해야 합니다:
1) RPACA 파일을 저장 및 가져오기 할 폴더를 선택합니다.
2) 필요한 작업을 수행하기 위해 필요한 명령어를 작성합니다.
3) 작성한 명령어를 RPACA 파일에 저장합니다.
4) RPACA

In [36]:
result = stream("반드시 prompt 내용만을 사용하여 '질문-답변'을 5개 이상 만들어주세요.", model, system_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. 질문: RPACA 설치 가이드에서 어떻게 설치를 시작해야 하나요?
 답변: 1)RPACA 설치파일을 더블 클릭합니다. 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. 3)Install 버튼을 클릭합니다. 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.
2. 질문: RPACA 사용법에서 어떻게 업무자동화를 만들어야 하나요?
 답변: 1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다. 2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. 3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다. 4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다. 5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다. 6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다. (주모니터에서만 작동) 7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.
3. 질문: RPACA 사용법에서 어떻게 이미지를 캡처하는 방법인가요?
 답변: 1)이름이 Image로 끝나는 Activity를 클릭하면 Option 옆에 사진기 아이콘이 존재합니다. 해당아이콘을 클릭하면 알파카가 최소화 되면서 원하는 영역을 드래그 앤 드롭으로 지정하면 해당 영역이 캡처됩니다. (예시 이미지에선 검색 아이콘을 지정)
4. 질문: RPACA 사용법에서 어떻게 캡처한 이미지를 기준으로 브라우저 내의 원하는 부분을 클릭하는 방법인가요?
 답변: 1)이미지 캡처는 옵션쪽의 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다. Option
  imagePath : 사진기 아이콘을 클릭하고 이미지를 캡처하였을 때 해당 
 이미지를 문자화 시킨 값이 자동으로 들어갑니다. (신경 안쓰셔도 됩니다.)
5. 질문: RPACA 사용법

In [6]:
result = stream("반드시 prompt만을 사용하여 짧은 질문과 답변 형태를 10개 이상 만들어주세요.", model, system_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. RPACA는 무엇인가요?
RPACA는 사용자가 원하는 작업을 자동화하기 위한 스크립트 파일입니다. 이 스크립트 파일은 크롬 브라우저를 통해 실행되며, 사용자가 원하는 작업을 수행하기 위해 필요한 명령어를 포함하고 있습니다.
2. RPACA를 설치하기 위해서는 어떤 단계를 거쳐야 하나요?
RPACA를 설치하기 위해서는 다음과 같은 단계를 거쳐야 합니다:
1) RPACA 설치파일을 더블 클릭합니다.
2) Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다.
3) Install 버튼을 클릭합니다.
4) 설치가 다 진행 될때까지 기다린 후 FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.
3. RPACA를 사용하기 위해서는 어떤 단계를 거쳐야 하나요?
RPACA를 사용하기 위해서는 다음과 같은 단계를 거쳐야 합니다:
1) RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다.
2) 해당 폴더 경로가 표시된 상단 부분에 있는 파일 리스트에 RPACA 파일이 존재한다면 이미지 캡처 및 클릭 등의 작업을 수행할 수 있습니다.
3) 필요한 작업을 수행하기 위해 필요한 명령어를 작성하고 실행 버튼을 클릭합니다.
4) 작업이 완료되면 결과를 확인할 수 있습니다.
4. RPACA를 사용하기 위해서는 어떤 명령어를 사용해야 하나요?
RPACA를 사용하기 위해서는 다양한 명령어를 사용할 수 있습니다. 예를 들어, 크롬 브라우저를 실행하는 명령어는 openBrowser 명령어를 사용할 수 있습니다. 또한, 브라우저 내에서 특정 작업을 수행하는 명령어도 제공됩니다. 이러한 명령어를 사용하여 원하는 작업을 자동화할 수 있습니다.
5. RPACA를 사용하기 위해서는 어떤 설정을 해야 하나요?
RPACA를 사용하기 위해서는 다음과 같은 설정을 해야 합니다:
1) RPACA 파일을 저장 및 가져오기 할 폴더를 선택합니다.
2) 필요한 작업을 수행하기 위해 필요한 명령어를 작성합니다.
3) 작성한 명령어를 RPACA 파일에 저장합니다.
4) RPACA

In [15]:
result = stream("RPACA 설치 가이드에 대해 설명해주세요.", model, system_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RPACA 설치 가이드는 다음과 같은 단계로 진행됩니다:

1) RPACA 설치파일을 더블 클릭합니다.
2) Install 버튼이 나오기전까지 계속해서 Next 버튼을 클릭합니다.
3) Install 버튼을 클릭합니다.
4) 설치가 다 진행될 때까지 기다린 후 FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.

RPACA 설치 가이드에서는 설치 파일을 실행하고, 설치 단계를 따라가며 다음 단계로 진행합니다. 설치가 완료되면 바탕화면에 RPACA가 설치되었는지 확인할 수 있습니다.


In [5]:
result = stream('반드시 prompt의 내용만을 사용하여 짧은 질문과 답변을 10개 생성해주세요. 출력 형태는 {"instruction": "질문", "input": "", "output": "답변"} 같았으면 좋겠습니다.', model, system_prompt)

Token indices sequence length is longer than the specified maximum sequence length for this model (2761 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{"instruction": "RPA 자동화 프로그램을 설치하고 사용하는 방법을 알려주세요.", "input": "", "output": "RPA 자동화 프로그램을 설치하고 사용하기 위해서는 RPACA 설치파일을 다운로드하여 설치해야 합니다. 설치 후에는 RPACA 파일을 저장 및 가져오기 할 폴더를 선택하고 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결하여 업무자동화를 진행할 수 있습니다."}
{"instruction": "RPA 자동화 프로그램에서 이미지를 캡처하는 방법을 알려주세요.", "input": "", "output": "이미지를 캡처하기 위해서는 이미지 캡처 기능을 사용해야 합니다. 이미지 캡처 기능은 옵션 하단부에 있는 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다."}
{"instruction": "RPA 자동화 프로그램에서 이미지를 캡처하는 방법을 알려주세요.", "input": "", "output": "이미지를 캡처하기 위해서는 이미지 캡처 기능을 사용해야 합니다. 이미지 캡처 기능은 옵션 하단부에 있는 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다."}
{"instruction": "RPA 자동화 프로그램에서 이미지를 캡처하는 방법을 알려주세요.", "input": "", "output": "이미지를 캡처하기 위해서는 이미지 캡처 기능을 사용해야 합니다. 이미지 캡처 기능은 옵션 하단부에 있는 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다."}
{"instruction": "RPA 자동화 프로그램에서 이미지를 캡처하는 방법을 알려주세요.", "input": "", "output": "이미지를 캡처하기 위해서는 이미지 캡처 기능을 사용해야 합니다. 이미지 캡처 기능은 옵션 하단부에 있는 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다."}
{"instruc

In [49]:
test = result.split('[/INST]')[1]
test1 = test.split('질문: ')
print(test1)
print()

for i in range(len(test1)) :
    if i > 0 :
        question, answer = test1[i].split('답변: ')
        print(question)
        print(answer)
        print()



# print()
# str

['1. ', 'RPACA 설치 가이드에서 어떻게 설치를 시작해야 하나요?\n  답변: 1)RPACA 설치파일을 더블 클릭합니다. 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. 3)Install 버튼을 클릭합니다. 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.\n2. ', 'RPACA 사용법에서 어떻게 업무자동화를 만들어야 하나요?\n  답변: 1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다. 2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. 3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다. 4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다. 5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다. 6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다. (주모니터에서만 작동) 7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.\n3. ', 'RPACA 사용법에서 어떻게 이미지를 캡처하는 방법인가요?\n  답변: 1)이름이 Image로 끝나는 Activity를 클릭하면 Option 옆에 사진기 아이콘이 존재합니다. 해당아이콘을 클릭하면 알파카가 최소화 되면서 원하는 영역을 드래그 앤 드롭으로 지정하면 해당 영역이 캡처됩니다. (예시 이미지에선 검색 아이콘을 지정)\n4. ', 'RPACA 사용법에서 어떻게 캡처한 이미지를 기준으로 브라우저 내의 원하는 부분을 클릭하는 방법인가요?\n  답변: 1)이미지 캡처는 옵션쪽의 사진기 아이콘을 클릭하고 드래그 앤 드롭으로 원하는 영역을 지정하면 가능합니다. Option\n   imagePath : 사진기 아이콘을 클릭하고 이미지를 캡처하였을 때 해당 \n  이미지를 문자화 시킨 값이 자동으로 들어갑니다. (신경 안쓰셔도 됩니다.)

In [66]:
import json

data = {}
path = "/workspace/sample.json"

data['data'] = []
data['data'].append({
    str
})

data

{'data': [{'{"instruction": "RPACA 설치 가이드", "input": "", "output": "1)RPACA 설치파일을 더블 클릭합니다. 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. 3)Install 버튼을 클릭합니다. 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다."},{"instruction": "RPACA 사용법", "input": "", "output": "1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다. 2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. 상단 부분에는 선택한 폴더 경로가 표시됩니다. 3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다. 4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다. 5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다. 6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다. (주모니터에서만 작동) 7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다. Activity 이름이 Image로 끝날 경우 이미지 캡쳐하는 방법"},'}]}

In [12]:
test = result.split('[/INST]')[1]
test1 = test.split('\n')
test1 = [v for v in test1 if v]
test1

str = ""
for i in range(len(test1)) :
    if i % 2 == 0 : 
        a = test1[i].split('.')[1]
        print(f'{a.strip()}')
        str += a.strip()
        print(f'{test1[i+1].strip()}')
        str += test1[i+1].strip()
    else : continue

질문: RPACA 설치 가이드에서 어떤 단계에서 설치가 완료되었는지 확인할 수 있나요?
답변: 설치가 완료되면 FInish 버튼을 클릭하여 바탕화면에 설치되었는지 확인할 수 있습니다.
질문: RPACA 사용법에서 어떻게 업무자동화를 시작할 수 있나요?
답변: 업무자동화를 시작하기 위해서는 먼저 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결한 후 시작 버튼을 클릭합니다.
질문: RPACA 사용법에서 어떻게 이미지를 캡처할 수 있나요?
답변: 이미지를 캡처하기 위해서는 이름이 Image로 끝나는 Activity를 클릭하고 옵션 옆에 사진기 아이콘을 클릭하여 원하는 영역을 드래그 앤 드롭으로 지정하면 됩니다.
질문: RPACA 사용법에서 어떻게 캡처한 이미지를 사용할 수 있나요?
답변: 캡처한 이미지를 사용하기 위해서는 이미지 캡처 후 해당 이미지를 브라우저 내의 원하는 부분에 입력하거나 클릭할 수 있습니다.
질문: RPACA 사용법에서 어떻게 웹페이지의 특정 부분을 클릭할 수 있나요?
답변: 웹페이지의 특정 부분을 클릭하기 위해서는 이름이 XPath로 끝나는 Activity를 클릭하고 옵션 옆에 브라우저 내에서 해당 부분을 클릭하고 캡처한 이미지를 사용하여 클릭할 수 있습니다.
질문: RPACA 사용법에서 어떻게 웹페이지의 특정 부분을 입력할 수 있나요?
답변: 웹페이지의 특정 부분을 입력하기 위해서는 이름이 XPath로 끝나는 Activity를 클릭하고 옵션 옆에 브라우저 내에서 해당 부분을 입력하고 캡처한 이미지를 사용하여 입력할 수 있습니다.
질문: RPACA 사용법에서 어떻게 웹페이지의 특정 부분을 클릭하고 입력하는 동시에 캡처한 이미지를 사용할 수 있나요?
답변: 웹페이지의 특정 부분을 클릭하고 입력하는 동시에 캡처한 이미지를 사용하기 위해서는 이름이 XPath로 끝나는 Activity를 클릭하고 옵션 옆에 브라우저 내에서 해당 부분을 클릭하고 캡처한 이미지를 사용하여 입력할 수 있습니다.
질문: 

In [14]:
import json

data = {}
path = "/workspace/sample.json"

data['data'] = []
data['data'].append({
    str
})

with open(path, 'w') as outfile:
    json.dump(data, outfile, indent=4)

TypeError: Object of type set is not JSON serializable

In [25]:
!pip install jsonlines


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [30]:
ad = []
with open("/workspace/data/seed_tasks_test.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        ad.append(line.strip())

In [31]:
ad

['RPACA 설치 가이드  1)RPACA 설치파일을 더블 클릭합니다. \\n 2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다. \\n 3)Install 버튼을 클릭합니다. \\n 4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.',
 'RPACA 사용법  1)RPACA 파일을 저장 및 가져오기 할 폴더를 선택하여 클릭합니다.\\n2)해당 폴더 경로에 RPACA 파일이 존재한다면 Files 리스트에 나타납니다. \\n상단 부분에는 선택한 폴더 경로가 표시됩니다\\n3)저장버튼을 눌러 RPACA 스크립트 조작 준비를 마칩니다.4)Activities 를 클릭하면 업무자동화에 필요한 activity 리스트가 보입니다.\\n5)필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 \\n단계의 activity를 연결합니다. \\n6)필요한activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 \\n진행되는지 확인합니다. (주모니터에서만 작동)\\n7)마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.',
 'Activity 이름이 Image로 끝날 경우 이미지 캡쳐하는 방법  1)이름이 Image로 끝나는 Activity를 클릭하면 Option 옆에 사진기 \\n아이콘이 존재합니다. 해당아이콘을 클릭합니다. \\n2)아이콘을 클릭하면 알파카가 최소화 되면서 원하는 영역을 드래그 앤 \\n드롭으로 지정하면 해당 영역이 캡처됩니다. (예시 이미지에선 검색 \\n아이콘을 지정)\\n3)imagePath 부분에 이미지를 문자화 시킨 값이 들어가고, 옵션 \\n하단부에 캡처한 이미지가 나타나고 작업을 완료합니다.',
 'Activity 이름이 XPath로 끝날 경우 xpath 값 구하는 방법  1)크롬 브라우저에서 키보드의 F12 를 클릭합니다.\\n2)이미지에 표시 된 부분을 클릭합니다. 3)브라우저 내에서 xpath를 구하고자 하는 곳에 마우스를 올리면 \\n이미지